In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random
from sklearn.metrics.pairwise import cosine_similarity
import math
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.tokenize import TweetTokenizer
import psycopg2 as pg2
import psycopg2.extras as pgex
this_host='34.211.59.66'
this_user='postgres'
this_password='postgres'
conn = pg2.connect(host = this_host, 
                        user = this_user,
                        password = this_password)

#sql_select = '''select created_at, location, tweet_content, cleaned_tweet, vector from tweets ORDER BY RANDOM() LIMIT 300000;'''
sql_select = '''select created_at, location, tweet_content, cleaned_tweet, hashtags from tweets;'''
cur = conn.cursor(cursor_factory=pgex.RealDictCursor)
cur.execute(sql_select)
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)
df.reset_index(inplace = True)

In [2]:
len(df)

550271

In [3]:
import pickle
!pip install redis
import redis
redis_ip = '34.211.59.66'
r = redis.StrictRedis(redis_ip)

In [ ]:
tfd_svd_pipe = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('svd',TruncatedSVD(n_components=25))
])
tfd = tfd_pipe.steps[0][1]
svd = tfd_pipe.steps[1][1]

In [10]:
tfidf_vectorizer = TfidfVectorizer(min_df = 1, stop_words = 'english')

In [11]:
tweet_tfidf_fit = tfidf_vectorizer.fit(df['cleaned_tweet'].values.astype('U'))
tweet_tfidf_transform = tfidf_vectorizer.fit_transform(df['cleaned_tweet'].values.astype('U'))
#tweet_tfidf_fit = tfidf_vectorizer.fit(df['hashtags'].values.astype('U'))
#tweet_tfidf_transform = tfidf_vectorizer.fit_transform(df['hashtags'].values.astype('U'))

In [12]:
tweet_tfidf_fit = pickle.dumps(tweet_tfidf_fit)
r.set('tweet_tfidf_fit', tweet_tfidf_fit)
tweet_tfidf_transform = pickle.dumps(tweet_tfidf_transform)
r.set('tweet_tfidf_fit_transform', tweet_tfidf_transform)

True

In [13]:
import pickle
import redis
redis_ip = '34.211.59.66'
r = redis.StrictRedis(redis_ip)
tweet_tfidf = pickle.loads(r.get('tweet_tfidf_fit_transform'))

In [14]:
tweet_tfidf 

<550271x155792 sparse matrix of type '<class 'numpy.float64'>'
	with 3166931 stored elements in Compressed Sparse Row format>

In [15]:
SVD = TruncatedSVD(n_components = 300)
SVD_model_fit = SVD.fit(tweet_tfidf)
r = redis.StrictRedis(redis_ip)
LSA = pickle.dumps(SVD_model_fit)
r.set('tweet_SVD_fit', LSA)

MemoryError: 

In [ ]:
r.keys()